In [1]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../Data/hindi_train.csv")
data

,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [3]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.15, random_state=42)

print("Training Set:")
print(train_data.shape)
print("Validation Set:")
print(val_data.shape)

Training Set:
(18041, 4)
Validation Set:
(3184, 4)


In [4]:
import re

def preprocess_tokenize(text):
    # for removing punctuation from sentencesc
    text = str(text)
    # text = re.sub(r'(\d+)', r'', text) # remove numbers
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '') #
    text = re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text = re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text = re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text = re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text = re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text = re.sub(r"[&©@#ø;.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    # text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    # text = re.sub("([a-zA-Z])",' ',str(text)).lower() #removing all the single characters
    text = re.sub("(\s+)",' ',str(text)).lower() #removing extra spaces
    return text

In [5]:
train_text = [preprocess_tokenize(text) for text in train_data['Article']]
train_summary = [preprocess_tokenize(text) for text in train_data['Summary']]
val_text = [preprocess_tokenize(text) for text in val_data['Article']]
val_summary = [preprocess_tokenize(text) for text in val_data['Summary']]

In [6]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "csebuetnlp/mT5_m2o_hindi_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
input_text = "summarize: भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष सोनिया गांधी ने अपने आदेश दिए हैं कि वे अपने पद से इस्तीफा देने के लिए तैयार हैं।"
label_text = "सोनिया गांधी ने अपने पद से इस्तीफा देने की घोषणा की है।"
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
label_ids = tokenizer(label_text, return_tensors="pt", truncation=True).to(device)

outputs = model(input_ids=input_ids.input_ids, labels=label_ids.input_ids, attention_mask=input_ids.attention_mask, decoder_attention_mask=label_ids.attention_mask)
print(outputs.logits.shape, outputs.encoder_last_hidden_state.shape)
print(tokenizer.decode(outputs.logits[0].argmax(dim=-1)))
print(tokenizer.decode(label_ids.input_ids[0]))
print(outputs.loss)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([1, 22, 250112]) torch.Size([1, 45, 768])
निया गांधी के ने पद से इस्तीफा दे  की पेषणा कर है.</s>
सोनिया गांधी ने अपने पद से इस्तीफा देने की घोषणा की है।</s>
tensor(1.1835, device='cuda:0', grad_fn=<NllLossBackward0>)


In [8]:
input_text = "summarize: " + train_text[0]
label_text = train_summary[0]
input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
label_ids = tokenizer(label_text, return_tensors="pt", max_length=512, truncation=True).to(device)

outputs = model(input_ids=input_ids.input_ids, labels=label_ids.input_ids, attention_mask=input_ids.attention_mask, decoder_attention_mask=label_ids.attention_mask)
print(outputs.logits.shape, outputs.encoder_last_hidden_state.shape)
print(tokenizer.decode(outputs.logits[0].argmax(dim=-1)))
print(tokenizer.decode(label_ids.input_ids[0]))
print(outputs.loss)

torch.Size([1, 88, 250112]) torch.Size([1, 512, 768])
ए कीअंतरने एउ ए विइंडको ए मा लि ंस ए (  ए में केर-दराज क्षेत्रलाकों  हतर एयर कनेक्टिविटी बनाने के लिए एक इन इंडिया के विहत यरब्रेयर और सुस की कखोई कजै ग्लोबल एयरक्राफ्ट कंपनी के साथ मिलनरशिप</s></s></s></s>
india looks partner sukhoi embraer to manufacture small civilian plane locally भारत सरकार दूर-दराज इलाकों से बेहतर एयर कनेक्टिविटी बनाने के लिए मेक इन इंडिया के तहत एम्ब्रायर और रूस की सुखोई सहित ग्लोबल एयरक्राफ्ट कंपनी के साथ पार्टनरशिप करेगी।</s>
tensor(3.9363, device='cuda:0', grad_fn=<NllLossBackward0>)


In [9]:
input_text = "summarize: भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष सोनिया गांधी ने अपने आदेश दिए हैं कि वे अपने पद से इस्तीफा देने के लिए तैयार हैं।"
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True)["input_ids"].to(device)
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
summary

'कांग्रेस अध्यक्ष सोनिया गांधी ने अपने पद से इस्तीफा दे दिया है.'

In [10]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [12]:
import torch
import torch.nn as nn

class FFTActivation(nn.Module):
    def __init__(self):
        super(FFTActivation, self).__init__()
        self.fft = torch.fft.rfft

    def forward(self, x):
        return self.fft(x)

class SummarizartionModel(nn.Module):
    def __init__(self, model, tokenizer):
        super(SummarizartionModel, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.fft = FFTActivation()

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None):
        out = self.model(input_ids=input_ids, labels=decoder_input_ids, 
                attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask)
        out.encoder_last_hidden_state = self.fft(out.encoder_last_hidden_state)
        out.logits = self.fft(out.logits)
        return out
        
    def generate(self, text, max_length=50):
        input_ids = self.tokenizer(text, return_tensors='pt', truncation=True).input_ids.to(device)
        attention_mask = self.tokenizer(text, return_tensors='pt', truncation=True).attention_mask.to(device)
        summary_ids = self.model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=2, length_penalty=0.5, no_repeat_ngram_size=2, early_stopping=True)
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
summary_model = SummarizartionModel(model, tokenizer).to('cpu')

input_text = "भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष सोनिया गांधी ने अपने आदेश दिए हैं कि वे अपने पद से इस्तीफा देने के लिए तैयार हैं।"
label_text = "सोनिया गांधी ने अपने पद से इस्तीफा देने की घोषणा की है।"
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True)
label_ids = tokenizer(label_text, return_tensors="pt", truncation=True)

outputs = summary_model(input_ids=input_ids.input_ids, attention_mask=input_ids.attention_mask, 
                decoder_input_ids=label_ids.input_ids, decoder_attention_mask=label_ids.attention_mask)
                
print(outputs.logits.shape, outputs.encoder_last_hidden_state.shape)
# outputs.logits = outputs.logits.abs()
# print(tokenizer.decode(outputs.logits[0].argmax(dim=-1)))
print(tokenizer.decode(label_ids.input_ids[0]))
print(outputs.loss)

torch.Size([1, 22, 125057]) torch.Size([1, 42, 385])
सोनिया गांधी ने अपने पद से इस्तीफा देने की घोषणा की है।</s>
tensor(1.1900, grad_fn=<NllLossBackward0>)


In [24]:
input_text = "भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष सोनिया गांधी ने अपने आदेश दिए हैं कि वे अपने पद से इस्तीफा देने के लिए तैयार हैं।"
summary_model = summary_model.to(device)
summary = summary_model.generate(input_text)
summary

'कांग्रेस अध्यक्ष सोनिया गांधी ने अपने पद से इस्तीफा दे दिया है.'

In [25]:
from tqdm import tqdm

def generate_summaries(texts_to_summarize, tokenizer, model, data_name):
    generated_summary = []
    model = model.to(device)

    for i, input_text in tqdm(enumerate(texts_to_summarize), total=len(texts_to_summarize), desc="Processing Texts"):
        summary = model.generate('summarize:' + input_text)
        generated_summary.append(summary)
    print('summary generated for ', data_name)
    return generated_summary

val_generated_summary = generate_summaries(val_text, tokenizer, summary_model, "Validation Data") 

Processing Texts: 100%|██████████| 3184/3184 [35:06<00:00,  1.51it/s]

summary generated for  Validation Data


In [26]:
len(val_text), len(val_summary), len(val_generated_summary)

(3184, 3184, 3184)

In [27]:
from rouge import Rouge
import pandas as pd

rouge = Rouge()
scores = rouge.get_scores(val_generated_summary, val_summary, avg=True)
# rouge_scores = pd.DataFrame(scores).set_index(pd.Index(['rouge-1', 'rouge-2', 'rouge-l']))
rouge_scores = pd.DataFrame(scores).set_index([['recall','precision','f-measure']])
rouge_scores

,rouge-1,rouge-2,rouge-l
recall,0.200356,0.070905,0.165015
precision,0.352540,0.133151,0.289910
f-measure,0.249166,0.089686,0.205036


In [28]:
import plotly.express as px

fig = px.bar(rouge_scores*100, x=rouge_scores.index, y=rouge_scores.columns,
             barmode='group',
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()

In [29]:
from bert_score import score

def calculate_bert_score(predicted_corpus, reference_corpus):
    p, r, f1 = score(predicted_corpus, reference_corpus, lang="de")
    return p.mean().item(), r.mean().item(), f1.mean().item()

bert_score_val = calculate_bert_score(val_generated_summary, val_summary)
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_val[0]}')
print(f'Recall in BERT Score: {bert_score_val[1]}')
print(f'F1 Score in BERT Score: {bert_score_val[2]}')

BERT Scores:
Precision in BERT Score: 0.7342828512191772
Recall in BERT Score: 0.6820363402366638
F1 Score in BERT Score: 0.7067237496376038


In [30]:
[x for x in val_summary][:5]

['उत्तर प्रदेश के मथुरा जिले में यमुना एक्सप्रेस-वे पर आधा दर्जन बदमाशों ने दिल्ली से हमीरपुर जा रही निजी बस में सवारी के रूप में चढ़कर उसे बंधक बना लिया और तकरीबन सभी सवारियों से लाखों रुपये का सामान लूट किया।',
 'शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की मौजूदगी में मंत्री सुरेश राणा के पिता ठाकुर रणवीर सिंह का अंतिम संस्कार किया गया।',
 'up bhadohi bahubali ex-mla vijay mishra इस वक्त विजय मिश्रा, उनका बेटा विष्णु मिश्रा, भतीजा मनीष मिश्रा जेल में बंद हैं। इन सबकी करोड़ों रुपए की संपत्ति जब्त हो चुकी है।',
 'पिछले दो महीनों से मैं ये लगातार कह रहा हूं कि किसानों के आंदोलन में असामाजिक तत्वों और राजनीतिक कार्यकर्ताओं ने घुसपैठ की है।',
 'आम आदमी पार्टी ने पहले भारत की स्वतंत्रता की 75वीं वर्षगांठ पर इस तरह की यात्राएं लखनऊ, आगरा और नोएडा में आयोजित की थीं।']

In [31]:
[x for x in val_generated_summary][:5]


['उत्तर प्रदेश के मथुरा ज़िले में यमुना एक्सप्रेस-वे पर हुए हमले में कम से कम बीस लोगों की मौत हो गई है.',
 'उत्तर प्रदेश के मंत्री सुरेश राणा का शुक्रवार को निधन हो गया.',
 'उत्तर प्रदेश में विधानसभा चुनावों के नतीजे अभी तक सामने नहीं आए हैं.',
 'गणतंत्र दिवस पर किसानों की हिंसा और उनके विरोध को लेकर अब तक सवाल उठने लगे हैं.',
 'आम आदमी पार्टी ने कहा है कि वो उत्तर प्रदेश विधानसभा चुनावों में भगवान राम के दर्शन करेगी.']